In [4]:
#libraries
import os
import pandas as pd
from datetime import datetime as dt
from datetime import time as t
import numpy as np

In [5]:
# source
## Hoang
eps_path = r"C:\Users\dinhhoang.nguyen\OneDrive - Concentrix Corporation\WFM-internal\Data\EPS Tableau\EPS Query"
cpi_path = r"C:\Users\dinhhoang.nguyen\OneDrive - Concentrix Corporation\WFM-internal\Data\Ticket\CPI Query"
aht_path = r"C:\Users\dinhhoang.nguyen\OneDrive - Concentrix Corporation\WFM-internal\Data\AHT\AHT query"
csat_path = r"C:\Users\dinhhoang.nguyen\OneDrive - Concentrix Corporation\WFM-internal\Data\CSAT RAW\CSAT Query"
cuic_path = r"C:\Users\dinhhoang.nguyen\OneDrive - Concentrix Corporation\WFM-internal\Data\CUIC\CUIC Query"
exception_path = r"C:\Users\dinhhoang.nguyen\OneDrive - Concentrix Corporation\WFM-internal\Data\Exception"
extension_path = r"C:\Users\dinhhoang.nguyen\OneDrive - Concentrix Corporation\WFM-internal\Data\Extension"
iex_path = r"C:\Users\dinhhoang.nguyen\OneDrive - Concentrix Corporation\WFM-internal\Data\IEX"
## Tram
ioshrinkage_path = r"C:\Users\dinhhoang.nguyen\OneDrive - Concentrix Corporation\WFM-internal\Data\IO Shrinkage"
ot_path = r"C:\Users\dinhhoang.nguyen\OneDrive - Concentrix Corporation\WFM-internal\Data\OT\OT Query"
psat_path = r"C:\Users\dinhhoang.nguyen\OneDrive - Concentrix Corporation\WFM-internal\Data\PSAT RAW\PSAT Query"
quality_path = r"C:\Users\dinhhoang.nguyen\OneDrive - Concentrix Corporation\WFM-internal\Data\Quality.v2\Quality query"
ramco_path = r"C:\Users\dinhhoang.nguyen\OneDrive - Concentrix Corporation\WFM-internal\Data\Ramco\Ramco Query"
ramcoot_path = r"C:\Users\dinhhoang.nguyen\OneDrive - Concentrix Corporation\WFM-internal\Data\Ramco OT\Ramco OT Query"
requirement_path = r"C:\Users\dinhhoang.nguyen\OneDrive - Concentrix Corporation\WFM-internal\Data\Requirement"
## Thanh
schedule_path = r"C:\Users\dinhhoang.nguyen\OneDrive - Concentrix Corporation\WFM-internal\Data\Schedule 2\Schedule Query"
tl_path = r"C:\Users\dinhhoang.nguyen\OneDrive - Concentrix Corporation\WFM-internal\Data\TL"
training_path = r"C:\Users\dinhhoang.nguyen\OneDrive - Concentrix Corporation\WFM-internal\Data\Training"
masterroster_path = r"C:\Users\dinhhoang.nguyen\OneDrive - Concentrix Corporation\WFM-internal\Data\User"
workplan_path = r"C:\Users\dinhhoang.nguyen\OneDrive - Concentrix Corporation\WFM-internal\Data\Workplan"


In [6]:
#FUNCTION TO IMPORT FOLDERS
def import_csv(path):
    raw = []
    for file in os.listdir(path):
        if file.endswith(".csv"):
            file_path = os.path.join(path, file)
            file_data = pd.read_csv(file_path)
            file_data["filename"] = file
            raw.append(file_data)
            final_raw = pd.concat(raw, ignore_index=True)
    return final_raw
    
def import_xlsx(path, sheet_name):
    raw = []
    for file in os.listdir(path):
        if file.endswith(".xlsx"):
            file_path = os.path.join(path, file)
            file_data = pd.read_excel(file_path, engine="openpyxl", sheet_name = sheet_name)
            file_data["filename"] = file
            raw.append(file_data)
            final_raw = pd.concat(raw, ignore_index=True)
    return final_raw

In [2]:
# IMPORT MASTER ROSTER
masterroster = pd.read_excel(r"C:\Users\dinhhoang.nguyen\OneDrive - Concentrix Corporation\WFM-internal\Data\User\CNX Global Master Roster.xlsx", sheet_name="Sheet1")
masterroster['Employee_ID'] = masterroster['Employee_ID'].astype("Int64")

masterroster.info()

NameError: name 'pd' is not defined

In [8]:
# IMPORT EPS
eps = import_csv(path = eps_path)
eps = eps.drop(columns=['Index', 'filename', 'Date'])
eps['Session login VN'] = pd.to_datetime(eps['Session Login'], format ='mixed') + pd.Timedelta(hours=5)
eps['Session logout VN'] = pd.to_datetime(eps['Session Logout'], format ='mixed') + pd.Timedelta(hours=5)
eps['Session login date'] = pd.to_datetime(eps['Session login VN']).dt.date
eps['Session login time'] = pd.to_datetime(eps['Session login VN']).dt.time
eps['Session logout date'] = pd.to_datetime(eps['Session logout VN']).dt.date
eps['Session logout time'] = pd.to_datetime(eps['Session logout VN']).dt.time
eps = eps.drop(columns=['Session login VN', 'Session logout VN', 'Username', 'manager_username', 'sitecode', 'Session Time'])
eps.info()
eps.to_csv(r"C:\Users\dinhhoang.nguyen\OneDrive - Concentrix Corporation\WFM-internal\DB\filecsv\eps_test.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134315 entries, 0 to 134314
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   EID                  134315 non-null  int64 
 1   Session Login        134315 non-null  object
 2   Session Logout       134315 non-null  object
 3   BPE Code             134315 non-null  object
 4   Total Time           134315 non-null  int64 
 5   Session login date   134315 non-null  object
 6   Session login time   134315 non-null  object
 7   Session logout date  134315 non-null  object
 8   Session logout time  134315 non-null  object
dtypes: int64(2), object(7)
memory usage: 9.2+ MB


In [9]:
#IMPORT CPI
cpi = import_csv(path = cpi_path)
cpi = cpi.drop(columns=['Region Partner', 'Sitecode', 'Mp Csm Name', 'Mp Team Name'])
cpi['Date'] = cpi['filename'].replace(".csv", "", regex=True)
cpi['Date'] = pd.to_datetime(cpi['Date'], format = "mixed")


masterroster_cpi = masterroster[['Employee_ID', 'TED Name']]
cpi = pd.merge(cpi, masterroster_cpi, left_on="Staff Name", right_on="TED Name", how="left")
cpi = cpi.drop(columns=['filename', 'TED Name'])

cpi.info()
print(cpi)
cpi.to_csv(r"C:\Users\dinhhoang.nguyen\OneDrive - Concentrix Corporation\WFM-internal\DB\filecsv\cpi_test.csv")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54428 entries, 0 to 54427
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Staff Name         54428 non-null  object        
 1   Channel            41906 non-null  object        
 2   Number of Records  54428 non-null  int64         
 3   Date               54428 non-null  datetime64[ns]
 4   Employee_ID        53970 non-null  Int64         
dtypes: Int64(1), datetime64[ns](1), int64(1), object(2)
memory usage: 2.1+ MB
           Staff Name    Channel  Number of Records       Date  Employee_ID
0        Mikael Duong      phone                  4 1990-01-01    102208111
1            Kelly Le      phone                  1 1990-01-01    102207120
2       Jennie Nguyen      phone                  9 1990-01-01    102207121
3        Mikael Duong  messaging                  7 1990-01-01    102208111
4            Kelly Le  messaging                

In [16]:
#IMPORT AHT
aht = import_csv(path = aht_path)
aht = aht.drop(columns=['filename'])
masterroster_aht = masterroster[['Employee_ID', 'TED Name']]
aht = pd.merge(aht, masterroster_aht, left_on='Staff', right_on ='TED Name', how='left')
aht = aht.drop(columns=['TED Name'])
aht['Date'] = pd.to_datetime(aht['Date'], format='mixed')

aht.info()

print(aht)

C:\Users\dinhhoang.nguyen\AppData\Local\Temp\VSCodePortable-x64Temp\ipykernel_25144\2378756514.py:7: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  file_data = pd.read_csv(file_path)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 313932 entries, 0 to 313931
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   Date                  313932 non-null  datetime64[ns]
 1   Start Time            313932 non-null  object        
 2   End Time              313932 non-null  object        
 3   Staff                 313932 non-null  object        
 4   Language              234090 non-null  object        
 5   Item Channel          313932 non-null  object        
 6   Topic                 313932 non-null  object        
 7   Subtopic              313932 non-null  object        
 8   "Handling Time"       313932 non-null  object        
 9   First Email Id        0 non-null       float64       
 10  First Hotel Id        267584 non-null  object        
 11  First Reservation Id  257111 non-null  object        
 12  Tooltip Phone Time    67292 non-null   object        
 13 

In [15]:
#IMPORT CSAT
csat = import_csv(path = csat_path)
masterroster_csat = masterroster[['Employee_ID', 'TED Name']]


csat['Date '] = pd.to_datetime(csat['Date '], format='%m/%d/%Y')
csat = pd.merge(csat, masterroster_csat, left_on='Staff', right_on ='TED Name', how='left')

csat = csat.drop(columns=['Sort by Dimension', 'Has Comment', '"Comment"', 'Reservation Link', 'View comment', 'Sort by Dimension (copy)', 'filename', 'TED Name', 'Max. Sort by Dimension'])
print(csat)

        Survey Id  Reservation           Team    Channel          Staff  \
0      3018426701   3056847985   Duy Hoang Ho      email     Mia Nguyen   
1      3018613711   2665820829     Nina Huynh      email     Henry Phan   
2      3018623696   3548174839     Nina Huynh  messaging     Henry Phan   
3      3018646254   2314992465   Duy Hoang Ho  messaging     Ann Nguyen   
4      3018660236   3138227680     Nina Huynh      email     Naomi Tran   
...           ...          ...            ...        ...            ...   
19968  3378146389   4088395169     Nina Huynh      phone     Lizzy Dinh   
19969  3356856217   3586016392      An Nguyen      email       Vince Le   
19970  3373841198   3280129076      Kobe Tran      phone   Issac Nguyen   
19971  3346163735   3267877510  Phoebe Nguyen      phone  Celcelia Pham   
19972  3345078384   3364238064     Nina Huynh      phone       Gus Pham   

             Type      Date   \
0      touchpoint 2023-06-01   
1      touchpoint 2023-06-01   
2  

In [12]:
# IMPORT SCHEDULE
schedule = import_xlsx(path = schedule_path,sheet_name='Sheet1')
schedule['Attribute'] = pd.to_datetime(schedule['Attribute'],format='mixed').dt.date
schedule=schedule.iloc[:,0:8]
condition=(schedule['Value']=='OFF')|(schedule['Value']=='AL')|(schedule['Value']=='CO')|(schedule['Value']=='UPL')|(schedule['Value']=='Training')
schedule['Shift'] = np.where(condition, 'OFF', schedule['NS Check'])
schedule=schedule[['Emp ID','Name','Attribute','Value','LOB','Shift']]
schedule=schedule.rename(columns={'Attribute':'Date','Value':'Shift','Shift':'Shift_type'})

print(schedule)

          Emp ID          Name        Date      Shift           LOB Shift_type
0      102003080  Doan Ai Khue  2023-07-31  1400-2300  Senior VICSP         DS
1      102003080  Doan Ai Khue  2023-08-01        OFF  Senior VICSP        OFF
2      102003080  Doan Ai Khue  2023-08-02  1400-2300  Senior VICSP         DS
3      102003080  Doan Ai Khue  2023-08-03  1400-2300  Senior VICSP         DS
4      102003080  Doan Ai Khue  2023-08-04  1400-2300  Senior VICSP         DS
...          ...           ...         ...        ...           ...        ...
29709  102269110   Lê Phúc Thọ  2023-09-27  1300-2200            EN         DS
29710  102269110   Lê Phúc Thọ  2023-09-28  1300-2200            EN         DS
29711  102269110   Lê Phúc Thọ  2023-09-29  1300-2200            EN         DS
29712  102269110   Lê Phúc Thọ  2023-09-30        OFF            EN        OFF
29713  102269110   Lê Phúc Thọ  2023-10-01        OFF            EN        OFF

[29714 rows x 6 columns]


In [17]:
#IMPORT CUIC
cuic = import_xlsx(path = cuic_path,sheet_name="Khoa Interval Attendance Check-")
cuic['Date'] = pd.to_datetime(cuic['Interval'], format = "mixed").dt.date
cuic['Time'] = pd.to_datetime(cuic['Interval'], format = "mixed").dt.time
cuic = pd.merge(cuic, masterroster, left_on='LoginName', right_on ='Booking Login ID', how='left')
cuic=cuic.iloc[:,0:9]
cuic = pd.merge(cuic, schedule, left_on=['Employee_ID','Date'], right_on =['Emp ID','Date'], how='left')
cuic=cuic[['FullName','LoginName','Interval','AgentAvailTime','AgentLoggedOnTime','Date','Time','Employee_ID','Shift_type','Shift']]
cuic['Date-1']=pd.to_datetime(cuic['Date'], format ='mixed') - pd.Timedelta(1,"d")
cuic['Date-1']=pd.to_datetime(cuic['Date-1'], format ='mixed').dt.date
cuic = pd.merge(cuic, schedule, left_on=['Employee_ID','Date-1'], right_on =['Emp ID','Date'], how='left')
cuic=cuic[['FullName','LoginName','Interval','AgentAvailTime','AgentLoggedOnTime','Date_x','Time','Employee_ID','Shift_type_x','Shift_x','Date-1','Shift_type_y','Shift_y']]
mytime=t(12,0,0)
condition=(cuic['Shift_type_x']!='DS')&(cuic['Shift_type_y']=='NS')&(cuic['Time']<mytime)
cuic['Session Date'] = np.where(condition, cuic['Date-1'], cuic['Date_x'])
cuic=cuic[['FullName','LoginName','Interval','AgentAvailTime','AgentLoggedOnTime','Date_x','Employee_ID','Shift_type_x','Session Date']]
cuic=cuic.rename(columns={'Date_x':'Date','Shift_type_x':'Shift_type'})

: 

In [ ]:
#EXCEPTION
exceptional_hrs = import_xlsx(path= exception_path, sheet_name="Sheet1")
exceptional_hrs = exceptional_hrs.rename(columns={'Date \n(MM/DD/YYYY)':'Date'})
exceptional_hrs = exceptional_hrs.drop(columns=['filename'])

exceptional_hrs.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Emp ID                       8 non-null      int64         
 1   Date                         8 non-null      datetime64[ns]
 2   Exception request 
(Minute)  8 non-null      int64         
 3   Reason                       8 non-null      object        
 4   TL                           8 non-null      object        
 5   OM                           8 non-null      object        
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 516.0+ bytes


In [ ]:
#IMPORT EXTENSION
extension = import_xlsx(path= extension_path, sheet_name="Sheet1")
extension = extension[['Emp ID', 'Extension Number']]
extension['Extension Number'] = extension['Extension Number'].astype("Int64")
print(extension)


        Emp ID  Extension Number
0    101754274          72786027
1    101758902          72786060
2    101758951          72786058
3    101758868          72786059
4    101631001          72786086
..         ...               ...
524  102278515          72786990
525  102278516          72786860
526  102279846          72786973
527  102279847          72786988
528  102279848          72786563

[529 rows x 2 columns]


In [ ]:
#IMPORT IEX ID
iex = import_xlsx(path = iex_path, sheet_name="Sheet1")
iex = iex[['ID Agent', 'Personal ID']]
iex = iex.rename(columns={'Personal ID': 'EID'})
print(iex)

     ID Agent        EID
0     3006158  101988331
1     3009891  101754192
2     3011158  101631001
3     3014249  101769330
4     3014252  101769328
..        ...        ...
587   3069722  102278512
588   3069723  102278397
589   3069755  102279847
590   3069759  102279846
591   3069760  102279848

[592 rows x 2 columns]


In [ ]:
# IMPORT TL
tl = pd.read_excel(os.path.join(tl_path, "TL List.xlsx"),sheet_name='Sheet1')
tl=tl.iloc[:,0:6]
tl=tl.rename(columns={'Supervisor (no need fill)': 'Supervisor'})
tl=tl.drop(columns=['Wave', 'Name', 'LOB', 'Full name (VN)'])
print(tl)

        Emp ID Supervisor
0    101754274       Nina
1    101758868       Nina
2    101993685   John Luu
3    101993760   John Luu
4    102177117         An
..         ...        ...
594  102265696        Gio
595  102265697       Rosa
596  102267302        Gio
597  102267344       Rosa
598  102267481        Gio

[599 rows x 2 columns]


In [ ]:
# IMPORT TRAINING
training = import_xlsx(path = training_path,sheet_name='Sheet1')
training['Date'] = pd.to_datetime(training['Date'],format='mixed').dt.date
training['EID'] = training['EID'].astype("int64")
training=training.iloc[:,1:]

print(training)



           Date        EID    LOB  Duration
0    2023-05-16  102198234  VICSG       7.5
1    2023-05-16  101993685  VICSG       7.5
2    2023-05-16  101993770  VICSG       7.5
3    2023-05-16  102168268     NL       7.5
4    2023-05-16  102168217    ID4       7.5
..          ...        ...    ...       ...
177  2023-07-28  102273515     EN       3.0
178  2023-07-28  102273437     EN       3.0
179  2023-07-28  102273538     EN       3.0
180  2023-07-28  102277706     EN       3.0
181  2023-07-28  102274819     EN       3.0

[182 rows x 4 columns]


In [ ]:
# IMPORT IOSHRINKAGE
ioshrinkage = import_xlsx(path = ioshrinkage_path, sheet_name='Sheet1')

In [ ]:
# IMPORT OT

ot = import_xlsx(path = ot_path, sheet_name='Sheet2')

In [ ]:
# IMPORT PSAT
psat = import_csv(path = psat_path)
psat['Date'] = pd.to_datetime(psat['Date'],format='mixed').dt.date

In [ ]:
# IMPORT QUALITY
quality_raw = import_xlsx(path = quality_path,sheet_name='Sheet1')
quality_raw['eval_date'] = pd.to_datetime(quality_raw['eval_date'],format='mixed').dt.date
quality=quality_raw.iloc[:,0:20]

In [ ]:
# IMPORT RAMCO
ramco = import_csv(path = ramco_path)
ramco['Attribute'] = pd.to_datetime(ramco['Attribute'],format='mixed').dt.date
ramco=ramco.rename(columns={'Attribute':'Date'})
ramco['EID']=ramco['EID'].astype('Int64')

In [ ]:
# IMPORT RAMCOOT
ramcoot = import_xlsx(path = ramcoot_path,sheet_name='Sheet1')
ramcoot['Attribute'] = pd.to_datetime(ramcoot['Attribute'],format='mixed').dt.date
ramcoot=ramcoot.rename(columns={'Attribute':'Date'})


In [ ]:
# IMPORT REQUIREMENT
requirement = import_xlsx(path = requirement_path,sheet_name='Sheet1')
requirement['Date'] = pd.to_datetime(requirement['Date'],format='mixed').dt.date

In [ ]:
print(cuic)

          FullName LoginName            Interval  AgentAvailTime  \
0         Ta, Kyle       kta  8/1/23 10:00:00 AM        0.018021   
1         Ta, Kyle       kta  8/1/23 10:30:00 AM        0.001319   
2         Ta, Kyle       kta  8/1/23 11:00:00 AM        0.011227   
3         Ta, Kyle       kta  8/1/23 11:30:00 AM        0.010752   
4       Bach, Tina     tbach   8/1/23 6:30:00 AM        0.000000   
...            ...       ...                 ...             ...   
317424  Vu, Olivia       ovu  9/19/23 5:00:00 AM        0.000000   
317425  Vu, Olivia       ovu  9/19/23 5:30:00 AM        0.000000   
317426  Vu, Olivia       ovu  9/19/23 6:00:00 AM        0.000000   
317427  Vu, Olivia       ovu  9/19/23 6:30:00 AM        0.000000   
317428  Vu, Olivia       ovu  9/19/23 7:00:00 AM        0.000000   

        AgentLoggedOnTime        Date  Employee_ID Shift_type Session Date  
0                0.019769  2023-08-01    102003107         DS   2023-08-01  
1                0.020833  20

In [ ]:
#BOOKING ATTENDANCE
bkn_att = schedule.drop(columns=['Shift_type'])
bkn_att_ramco = ramco[['EID', 'Value']]
bkn_att = pd.merge(bkn_att, tl, on='Emp ID', how='left')
bkn_att = pd.merge(bkn_att, bkn_att_ramco, left_on ='Emp ID', right_on='EID')
bkn_att = bkn_att.drop(columns=['EID'])
bkn_att = bkn_att.rename(columns={'Value': 'Ramco'})

#bkn_att_cuic = pd.pivot_table(cuic, values =['AgentAvailTime', 'AgentLoggedOnTime'] , index=['Employee_ID', 'Session Date'], aggfunc=['sum'])
bkn_att_cuic = cuic[['Employee_ID', 'Session Date', 'AgentAvailTime', 'AgentLoggedOnTime']].groupby(['Employee_ID', 'Session Date']).sum()
print(bkn_att_cuic)
bkn_att = pd.merge(bkn_att, bkn_att_cuic, left_on=['Emp ID', 'Date'], right_on=['Employee_ID', 'Session Date'], how='left')


#bkn_att_cuic.to_csv(r'C:\Users\dinhhoang.nguyen\OneDrive - Concentrix Corporation\WFM-internal\Calculators\Joey Nguyen\py\cuic_test.csv')

NameError: name 'schedule' is not defined